# Strategy Comparison

This notebook compares multiple trading strategies on the same dataset to identify which performs best.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Any, Dict, List

from simple_backtest import Backtest, BacktestConfig
from simple_backtest.strategy.strategy_base import Strategy
from simple_backtest.strategy.buy_and_hold import BuyAndHoldStrategy
from simple_backtest.strategy.moving_average import MovingAverageStrategy
from simple_backtest.visualization.plotter import plot_equity_curve

## 1. Define All Strategies

In [ ]:
# RSI Strategy
class RSIStrategy(Strategy):
    def __init__(self, period: int = 14, oversold: float = 30, overbought: float = 70, shares: float = 100):
        super().__init__(name=f"RSI_{period}")
        self.period = period
        self.oversold = oversold
        self.overbought = overbought
        self.shares = shares
    
    def calculate_rsi(self, prices: pd.Series) -> float:
        if len(prices) < self.period + 1:
            return 50.0
        deltas = prices.diff()
        gains = deltas.where(deltas > 0, 0)
        losses = -deltas.where(deltas < 0, 0)
        avg_gain = gains.tail(self.period).mean()
        avg_loss = losses.tail(self.period).mean()
        if avg_loss == 0:
            return 100.0
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        rsi = self.calculate_rsi(data['Close'])
        has_position = False
        if trade_history:
            total_bought = sum(t['shares'] for t in trade_history if t['signal'] == 'buy')
            total_sold = sum(t['shares'] for t in trade_history if t['signal'] == 'sell')
            has_position = total_bought > total_sold
        if rsi < self.oversold and not has_position:
            return {"signal": "buy", "size": self.shares, "order_ids": None}
        elif rsi > self.overbought and has_position:
            return {"signal": "sell", "size": self.shares, "order_ids": None}
        return {"signal": "hold", "size": 0, "order_ids": None}


# Bollinger Bands Strategy
class BollingerBandsStrategy(Strategy):
    def __init__(self, window: int = 20, num_std: float = 2.0, shares: float = 100):
        super().__init__(name=f"BB_{window}_{num_std}")
        self.window = window
        self.num_std = num_std
        self.shares = shares
    
    def calculate_bollinger_bands(self, prices: pd.Series):
        if len(prices) < self.window:
            return None, None, None, prices.iloc[-1]
        middle_band = prices.tail(self.window).mean()
        std = prices.tail(self.window).std()
        upper_band = middle_band + (self.num_std * std)
        lower_band = middle_band - (self.num_std * std)
        return middle_band, upper_band, lower_band, prices.iloc[-1]
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        middle, upper, lower, current_price = self.calculate_bollinger_bands(data['Close'])
        if middle is None:
            return {"signal": "hold", "size": 0, "order_ids": None}
        has_position = False
        if trade_history:
            total_bought = sum(t['shares'] for t in trade_history if t['signal'] == 'buy')
            total_sold = sum(t['shares'] for t in trade_history if t['signal'] == 'sell')
            has_position = total_bought > total_sold
        if current_price <= lower and not has_position:
            return {"signal": "buy", "size": self.shares, "order_ids": None}
        elif current_price >= upper and has_position:
            return {"signal": "sell", "size": self.shares, "order_ids": None}
        return {"signal": "hold", "size": 0, "order_ids": None}

## 2. Download Market Data

In [ ]:
# Download S&P 500 data
ticker = "SPY"
start_date = "2019-01-01"
end_date = "2023-12-31"

data = yf.download(ticker, start=start_date, end=end_date)
print(f"Downloaded {len(data)} rows of {ticker} data")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.tail()

## 3. Run All Strategies

In [ ]:
# Configure backtest
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="percentage",
    commission_value=0.001,
    execution_price="open",
    parallel_execution=True,
    risk_free_rate=0.02,
)

# Create all strategies
strategies = [
    BuyAndHoldStrategy(shares=50),
    MovingAverageStrategy(short_window=10, long_window=30, shares=50),
    MovingAverageStrategy(short_window=20, long_window=50, shares=50),
    RSIStrategy(period=14, oversold=30, overbought=70, shares=50),
    BollingerBandsStrategy(window=20, num_std=2.0, shares=50),
]

# Run backtest
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

print("\nBacktest completed for all strategies!")

## 4. Compare Performance Metrics

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    name: res['metrics'] 
    for name, res in results.items() 
    if name != 'benchmark'
}).T

# Select key metrics
key_metrics = ['total_return', 'cagr', 'sharpe_ratio', 'sortino_ratio', 
               'max_drawdown', 'win_rate', 'total_trades', 'final_value']

print("\n=== Strategy Performance Comparison ===")
comparison_df[key_metrics].round(2)

In [ ]:
# Rank strategies by different metrics
print("\n=== Rankings by Sharpe Ratio ===")
print(comparison_df[['sharpe_ratio', 'total_return', 'max_drawdown']].sort_values('sharpe_ratio', ascending=False))

print("\n=== Rankings by Total Return ===")
print(comparison_df[['total_return', 'sharpe_ratio', 'max_drawdown']].sort_values('total_return', ascending=False))

## 5. Visualize Equity Curves

In [ ]:
# Plot all equity curves
fig = plot_equity_curve(results)
fig.show()

## 6. Risk-Return Analysis

In [ ]:
# Create risk-return scatter plot
plt.figure(figsize=(12, 7))

for name in comparison_df.index:
    plt.scatter(comparison_df.loc[name, 'volatility'], 
                comparison_df.loc[name, 'total_return'],
                s=150, alpha=0.6)
    plt.annotate(name, 
                (comparison_df.loc[name, 'volatility'], 
                 comparison_df.loc[name, 'total_return']),
                fontsize=9, ha='right')

plt.xlabel('Volatility (Annualized %)')
plt.ylabel('Total Return (%)')
plt.title('Risk-Return Profile of Strategies')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Drawdown Comparison

In [ ]:
# Plot drawdowns for all strategies
plt.figure(figsize=(14, 8))

for name, res in results.items():
    if name == 'benchmark':
        continue
    
    portfolio_values = res['portfolio_values']
    cumulative_max = portfolio_values.cummax()
    drawdown = (portfolio_values - cumulative_max) / cumulative_max * 100
    
    plt.plot(drawdown.index, drawdown, label=name, alpha=0.7)

plt.xlabel('Date')
plt.ylabel('Drawdown (%)')
plt.title('Drawdown Comparison Across Strategies')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Monthly Returns Heatmap

In [ ]:
# Calculate monthly returns for each strategy
monthly_returns = {}

for name, res in results.items():
    if name == 'benchmark':
        continue
    returns = res['returns']
    monthly = returns.resample('ME').sum() * 100  # Convert to percentage
    monthly_returns[name] = monthly

# Create DataFrame
monthly_df = pd.DataFrame(monthly_returns)

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
im = ax.imshow(monthly_df.T, aspect='auto', cmap='RdYlGn', vmin=-10, vmax=10)

# Set ticks and labels
ax.set_xticks(range(len(monthly_df.index)))
ax.set_xticklabels([d.strftime('%Y-%m') for d in monthly_df.index], rotation=45, ha='right')
ax.set_yticks(range(len(monthly_df.columns)))
ax.set_yticklabels(monthly_df.columns)

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Monthly Return (%)')

plt.title('Monthly Returns Heatmap')
plt.tight_layout()
plt.show()

## 9. Summary Statistics

In [ ]:
# Calculate additional statistics
summary = pd.DataFrame()

for name, res in results.items():
    if name == 'benchmark':
        continue
    
    returns = res['returns']
    metrics = res['metrics']
    
    summary.loc[name, 'Mean Daily Return (%)'] = returns.mean() * 100
    summary.loc[name, 'Std Daily Return (%)'] = returns.std() * 100
    summary.loc[name, 'Best Day (%)'] = returns.max() * 100
    summary.loc[name, 'Worst Day (%)'] = returns.min() * 100
    summary.loc[name, 'Positive Days (%)'] = (returns > 0).sum() / len(returns) * 100
    summary.loc[name, 'Sharpe Ratio'] = metrics['sharpe_ratio']
    summary.loc[name, 'Max Drawdown (%)'] = metrics['max_drawdown']

print("\n=== Summary Statistics ===")
summary.round(2)

## 10. Conclusion

Based on the analysis above, we can draw conclusions about:
- Which strategy provides the best risk-adjusted returns (Sharpe Ratio)
- Which strategy has the lowest drawdown
- Which strategy is most consistent (win rate)
- Trade-offs between different approaches

In [ ]:
# Find best strategy by different criteria
print("\n=== Best Strategies ===")
print(f"Highest Sharpe Ratio: {comparison_df['sharpe_ratio'].idxmax()} ({comparison_df['sharpe_ratio'].max():.2f})")
print(f"Highest Total Return: {comparison_df['total_return'].idxmax()} ({comparison_df['total_return'].max():.2f}%)")
print(f"Lowest Drawdown: {comparison_df['max_drawdown'].idxmin()} ({comparison_df['max_drawdown'].min():.2f}%)")
print(f"Highest Win Rate: {comparison_df['win_rate'].idxmax()} ({comparison_df['win_rate'].max():.2f}%)")